In [2]:
!pip install rouge-score 

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9979edbc04669f37184044f59c0e54eab4f6e70a17fd39f6ee4962ed325636f2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [3]:
import os
import torch
import random
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# Đọc dữ liệu từ file
with open('/kaggle/input/dataset/en_sents_dataset.txt', 'r', encoding='utf-8') as f:
    en = [line.strip() for line in f.readlines() if line.strip()]
with open('/kaggle/input/dataset/vi_sents_dataset.txt', 'r', encoding='utf-8') as f:
    vi = [line.strip() for line in f.readlines() if line.strip()]
assert len(en) == len(vi), "Số câu EN và VI không khớp!"

# Chia bộ dữ liệu thành Train/Val/Test
en_train, en_temp, vi_train, vi_temp = train_test_split(en, vi, test_size=0.2, random_state=42)
en_val, en_test, vi_val, vi_test = train_test_split(en_temp, vi_temp, test_size=0.5, random_state=42)

print(f"Train: {len(en_train)} | Val: {len(en_val)} | Test: {len(en_test)}")

Train: 597182 | Val: 74648 | Test: 74648


In [5]:
vocab_size = 30000
special_tokens = ['[PAD]', '[UNK]', '[BOS]', '[EOS]']

# Train tokenizer
def train_tokenizer(txt_file, save_path):
    tokenizer = Tokenizer(models.BPE())
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
    tokenizer.decoder = decoders.ByteLevel()
    trainer = trainers.BpeTrainer(vocab_size=vocab_size, special_tokens=special_tokens)
    tokenizer.train([txt_file], trainer)
    tokenizer.save(save_path)
    return tokenizer

# Tạo thư mục và train tokenizer cho tiếng Anh và tiếng Việt
os.makedirs("checkpoints", exist_ok=True)
tokenizer_src = train_tokenizer("/kaggle/input/dataset/en_sents_dataset.txt", "checkpoints/tokenizer_src.json")
tokenizer_tgt = train_tokenizer("/kaggle/input/dataset/vi_sents_dataset.txt", "checkpoints/tokenizer_tgt.json")

In [6]:
# Định nghĩa mô hình GPT
class GPTBlock(nn.Module):
    def __init__(self, dim, n_heads, ff_dim, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, n_heads, dropout=dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(dim)
        self.ff = nn.Sequential(
            nn.Linear(dim, ff_dim),
            nn.GELU(),
            nn.Linear(ff_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x, mask):
        x = x + self.attn(self.ln1(x), self.ln1(x), self.ln1(x), attn_mask=mask)[0]
        return x + self.ff(self.ln2(x))

class GPT2Translator(nn.Module):
    def __init__(self, vocab_size, dim=256, n_layers=4, n_heads=4, ff_dim=1024, max_len=128, dropout=0.1):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, dim)
        self.pos_emb = nn.Embedding(max_len, dim)
        self.blocks = nn.ModuleList([
            GPTBlock(dim, n_heads, ff_dim, dropout) for _ in range(n_layers)
        ])
        self.ln = nn.LayerNorm(dim)
        self.out = nn.Linear(dim, vocab_size)

    def forward(self, x):
        B, T = x.shape
        pos = torch.arange(0, T, device=x.device).unsqueeze(0)
        x = self.token_emb(x) + self.pos_emb(pos)
        mask = torch.triu(torch.ones(T, T, device=x.device), diagonal=1).bool()
        for block in self.blocks:
            x = block(x, mask)
        return self.out(self.ln(x))

In [7]:
# Khởi tạo thiết bị (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer_src = Tokenizer.from_file("checkpoints/tokenizer_src.json")
tokenizer_tgt = Tokenizer.from_file("checkpoints/tokenizer_tgt.json")
pad_id = tokenizer_tgt.token_to_id("[PAD]")
bos_id = tokenizer_tgt.token_to_id("[BOS]")
eos_id = tokenizer_tgt.token_to_id("[EOS]")

# Khởi tạo mô hình
model = GPT2Translator(vocab_size=tokenizer_tgt.get_vocab_size()).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

# Hàm mã hóa cặp câu (Bước tiền xử lý quan trọng)
def encode_pair(en_sent, vi_sent, max_len=128):
    # Tokenize câu tiếng Anh và tiếng Việt
    src_ids = tokenizer_src.encode(en_sent.lower()).ids[:max_len]
    tgt_ids = tokenizer_tgt.encode(vi_sent.lower()).ids[:max_len-1]
    
    # Thêm token BOS cho câu đích và EOS
    tgt_ids = [bos_id] + tgt_ids + [eos_id]
    tgt_ids = tgt_ids[:max_len]
    
    # Padding để đảm bảo chiều dài cố định
    src_ids += [pad_id] * (max_len - len(src_ids))
    tgt_ids += [pad_id] * (max_len - len(tgt_ids))
    
    return torch.tensor(src_ids), torch.tensor(tgt_ids)

# Hàm lấy batch
def get_batch(en_list, vi_list, batch_size=32):
    idxs = random.sample(range(len(en_list)), batch_size)
    srcs, tgts = [], []
    for i in idxs:
        src, tgt = encode_pair(en_list[i], vi_list[i])
        srcs.append(src)
        tgts.append(tgt)
    return torch.stack(srcs).to(device), torch.stack(tgts).to(device)

In [8]:
import torch
from tqdm import tqdm
import time

# Thiết bị huấn luyện
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Khởi tạo các biến cho early stopping
best_val_loss = float('inf')
no_improve_counter = 0
early_stopping_patience = 2  # Số lần cho phép không cải thiện trước khi dừng

batch_size = 32

# Huấn luyện mô hình
for epoch in range(5):
    model.train()
    total_loss = 0
    start_time = time.time()

    for _ in tqdm(range(len(en_train)//batch_size), desc=f"Epoch {epoch+1}"):
        src, tgt = get_batch(en_train, vi_train)
        src, tgt = src.to(device), tgt.to(device)

        logits = model(src)
        loss = criterion(logits.view(-1, logits.size(-1)), tgt.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    avg_train_loss = total_loss / (len(en_train)//batch_size)
    print(f"Epoch {epoch+1} - Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for _ in tqdm(range(len(en_val)//batch_size), desc="Validation"):
            src, tgt = get_batch(en_val, vi_val)
            src, tgt = src.to(device), tgt.to(device)

            logits = model(src)
            loss = criterion(logits.view(-1, logits.size(-1)), tgt.view(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / (len(en_val)//batch_size)
    print(f"Epoch {epoch+1} - Validation Loss: {avg_val_loss:.4f}")
    print(f"Epoch {epoch+1} - Time: {time.time() - start_time:.2f} sec")

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), f"checkpoints/gpt2_translator_epoch{epoch+1}.pth")
        no_improve_counter = 0
    else:
        no_improve_counter += 1
        if no_improve_counter >= early_stopping_patience:
            print(f"Early stopping after {epoch+1} epochs")
            break


Epoch 1: 100%|██████████| 18661/18661 [35:39<00:00,  8.72it/s]


Epoch 1 - Loss: 5.4953


Validation: 100%|██████████| 2332/2332 [01:44<00:00, 22.30it/s]


Epoch 1 - Validation Loss: 5.0072
Epoch 1 - Time: 2243.89 sec


Epoch 2: 100%|██████████| 18661/18661 [35:47<00:00,  8.69it/s]


...

Epoch 4 - Validation Loss: 4.2949
Epoch 4 - Time: 2252.49 sec


Epoch 5: 100%|██████████| 18661/18661 [35:48<00:00,  8.68it/s]


Epoch 5 - Loss: 4.1665


Validation: 100%|██████████| 2332/2332 [01:44<00:00, 22.35it/s]


Epoch 5 - Validation Loss: 4.2169
Epoch 5 - Time: 2253.32 sec


In [34]:
def generate_translation(model, src_sentence, max_len=128):
    model.eval()
    with torch.no_grad():
        src_ids = tokenizer_src.encode(src_sentence.lower()).ids[:max_len]
        src_ids += [pad_id] * (max_len - len(src_ids))
        src_tensor = torch.tensor([src_ids], device=device)

        # Bắt đầu với BOS token
        generated = [bos_id]
        for _ in range(max_len):
            tgt_tensor = torch.tensor([generated + [pad_id] * (max_len - len(generated))], device=device)
            logits = model(src_tensor)
            next_token = torch.argmax(logits[0, len(generated)-1]).item()

            # Dừng nếu gặp EOS
            if next_token == eos_id:
                break
            generated.append(next_token)

        # Giải mã thành câu
        decoded = tokenizer_tgt.decode(generated[1:])  # Bỏ BOS
        return decoded
# Dịch thử 15 câu đầu trong tập test
for i in range(15):
    src_sentence = en_test[i]
    ref_sentence = vi_test[i]
    pred_sentence = generate_translation(model, src_sentence)

    print(f"\nInput : {src_sentence}")
    print(f"Target: {ref_sentence}")
    print(f"Pred  : {pred_sentence}")



Input : Some people turn into rockers like this.
Target: Thực kinh khủng. Một vài người trở thành những rocker như thế.
Pred  :  một người người người vào đá như như như này

Input : As had so often happened on the Eastern Front Hitler refused to allow a strategic withdrawal until it was too late.
Target: "Tương tự như mặt trận phía đông, Hitler lại không cho phép rút quân cho đến khi quá trễ."
Pred  :  " như như thường ra ra ra phía phía phía hit phía phía chối chối phép cho cho rút chiến rút rút cho cho đến đến đến.

Input : "At 290 days old males are about 14.1 kg, and females are about 12.6 kg."
Target: "Khi 290 ngày tuổi, con đực đạt chừng 14,1 kg, còn con cái đạt khoảng 12.6 kg."
Pred  :  "vào 290 290 tuổi tuổi đực tuổi tuổi 14... kg, con con 12.6.666."."."

Input : "As a result, costs in the amount of 14 million euros were assumed."
Target: "Tuy nhiên, ước tính mức phí là khoảng 14 triệu euro."
Pred  :  "làết quả, phí phí trong lượng số triệu, 14 được được được được."."

Input 

In [35]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

smoothie = SmoothingFunction().method4
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
rougeLsum_scores = []

# Đánh giá trên 500 câu test đầu tiên
for i in range(500):
    src_sent = en_test[i]
    ref_sent = vi_test[i]
    pred_sent = generate_translation(model, src_sent)

    # Tính BLEU
    bleu = sentence_bleu(
        [ref_sent.split()],
        pred_sent.split(),
        smoothing_function=smoothie
    )
    bleu_scores.append(bleu * 100)

    # Tính ROUGE
    scores = scorer.score(ref_sent, pred_sent)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)
    rougeLsum_scores.append(scores['rougeLsum'].fmeasure)

# Trung bình các chỉ số
avg_bleu = np.mean(bleu_scores)
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)
avg_rougeLsum = np.mean(rougeLsum_scores)

# In kết quả
print("\n--- Evaluation Results ---")
print(f"BLEU: {avg_bleu:.2f}")
print(f"ROUGE-1: {avg_rouge1:.4f}")
print(f"ROUGE-2: {avg_rouge2:.4f}")
print(f"ROUGE-L: {avg_rougeL:.4f}")
print(f"ROUGE-Lsum: {avg_rougeLsum:.4f}")


--- Evaluation Results ---
BLEU: 6.64
ROUGE-1: 0.5054
ROUGE-2: 0.2493
ROUGE-L: 0.4181
ROUGE-Lsum: 0.4181
